In [ ]:
!pip install PyMuPDF pdfplumber matplotlib-venn geopandas osmnx openai shapely shap openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
#import libraries
import os
import re
import sys
import json
import time
import shap
import openai
import random
import pickle
import joblib
import hashlib
import warnings
import requests
import subprocess
import osmnx as ox
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from datetime import datetime
from osmnx import _overpass
from tabulate import tabulate
from google.colab import drive
from tabulate import tabulate
from IPython.display import display, HTML
from datetime import datetime, timedelta

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_loaded = pd.read_pickle('/content/drive/MyDrive/FYP /AI Alignment/Test/data/US_weather_report.pkl')

In [ ]:
print(tabulate(df_loaded.head(15), headers='keys', tablefmt='pretty'))

+----+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    |              Category               |                                                                                          Details                                                                                           |
+----+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 0  | WeatherForecastAndClimateConditions |                     HeavyRainfall > Locations > {'State': 'New Mexico', 'Details': 'Locally heavy precipitation sparking flash flooding', 'Date': 'October 19, 2024'}                      |
| 1  | WeatherForecastAndClimateConditions |                    

In [ ]:
def infer_climatic_zone(lat, lon):
    """
    Infer climatic zone based on latitude and longitude for the US.
    This is a simplified example based on broad climatic zone patterns.

    """
    if 24.396308 <= lat <= 49.384358 and -125.0 <= lon <= -66.93457:
        # Rough northern boundary for continental regions
        if lat > 40:
            return 'Continental Zone'
        # Central and temperate areas
        elif lat > 30:
            return 'Temperate Zone'
        # Lower latitudes, closer to subtropical and tropical regions
        elif lat <= 30:
            return 'Subtropical/Tropical Zone'
    else:
        return 'Unknown Zone'

# Initialize cache
location_cache = {}

def get_standardized_location_from_nominatim(location_name):
    # Check if location already cached
    if location_name in location_cache:
        return location_cache[location_name]

    # Call Nominatim API
    url = f"https://nominatim.openstreetmap.org/search?q={location_name}&format=json&addressdetails=1&limit=1"
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; MyPythonApp/1.0; +https://example.com)'}

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 0:
                location_data = data[0]
                address = location_data.get('address', {})

                # Extract standardized data
                state = address.get('state', 'N/A')
                county = address.get('county', 'N/A')
                city = address.get('town', '') or address.get('village', '') or address.get('city', location_name)

                # Extract latitude and longitude
                lat = float(location_data.get('lat', 0))
                lon = float(location_data.get('lon', 0))

                # Infer the climatic zone based on latitude and longitude
                climatic_zone = infer_climatic_zone(lat, lon)

                standardized_info = {
                    'state': state,
                    'county': county,
                    'city': city,
                    'climatic_zone': climatic_zone,
                    'full_standardized_name': f"{city}, {county}, {state}".strip(", ")
                }

                # Store location data in cache
                location_cache[location_name] = standardized_info
                return standardized_info
    except Exception as e:
        print(f"Error fetching location data for {location_name}: {e}")

    return None

In [ ]:
# Extract location from details in weather dataset
def extract_location(details):
    prompt = f"""Identify and extract the location names (state, county, city,
    or any relevant geographical area) mentioned in the following text:

    Text: "{details}"

    Extracted Locations:"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        location_response = response['choices'][0]['message']['content'].strip()
        return location_response

    except Exception as e:
        print(f"Error while extracting locations: {e}")
        return None

df_loaded['Extracted_Location'] = df_loaded['Details'].apply(extract_location)

In [ ]:
# Extract and categorize location information
def extract_location(details):
    #categorize location based on US geographical details
    prompt = f"""
    Based on the following text: "{details}",

    Identify and extract any of the following geographical details, if present:

    - State
    - County
    - City
    - Climatic Zone (e.g., Tropical, Subtropical, Temperate, Arid, Continental)

    Please return the information in the following format:
    State: <State Name or 'N/A'>
    County: <County Name or 'N/A'>
    City: <City Name or 'N/A'>
    Climatic Zone: <Climatic Zone Name or 'N/A'>
    Full Standardized Name: <Constructed Standardized Name or 'N/A'>
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        location_response = response['choices'][0]['message']['content'].strip()

        # Dictionary to store the parsed response
        standardized_info = {
            "state": "N/A",
            "county": "N/A",
            "city": "N/A",
            "climatic_zone": "N/A",
            "full_standardized_name": "N/A"
        }

        # Split response into lines and populate the standardized_info dictionary
        lines = location_response.splitlines()
        for line in lines:
            if "State:" in line:
                standardized_info['state'] = line.split(":", 1)[1].strip()
            elif "County:" in line:
                standardized_info['county'] = line.split(":", 1)[1].strip()
            elif "City:" in line:
                standardized_info['city'] = line.split(":", 1)[1].strip()
            elif "Climatic Zone:" in line:
                standardized_info['climatic_zone'] = line.split(":", 1)[1].strip()
            elif "Full Standardized Name:" in line:
                standardized_info['locations'] = line.split(":", 1)[1].strip()

        return standardized_info

    except Exception as e:
        print(f"Error while extracting locations: {e}")
        return {"state": "N/A", "county": "N/A", "city": "N/A", "climatic_zone": "N/A", "locations": "N/A"}

In [ ]:
df_loaded['Location_Info'] = df_loaded['Details'].apply(extract_location)

# json_normalize to expand the 'Location_Info' column into separate columns
df_loaded_expanded = pd.concat([df_loaded.drop(['Location_Info'], axis=1),
                                pd.json_normalize(df_loaded['Location_Info'])], axis=1)


df_final_output = df_loaded_expanded[['Category', 'Details', 'Extracted_Location', 'state', 'county', 'city', 'climatic_zone', 'locations']]

print("Weather Data with Expanded Location Information:")
print(tabulate(df_final_output, headers='keys', tablefmt='pretty'))

Weather Data with Expanded Location Information:
+----+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------+--------------+--------+------------+-----------------------------------------------------------------------------------+--------------------------------+
|    |              Category               |                                                                                          Details                                                                                           |                 Extracted_Location                  |    state     | county |    city    |                                   climatic_zone                                   |           locations            |
+----+-------------------------------------+-----------------------

In [ ]:
# Mapping of US state names to their codes
state_name_to_code = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO",
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH",
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND",
    "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA",
    "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
}

# get state code from state name
def get_state_code(state_name):
    return state_name_to_code.get(state_name, "N/A")

df_final_output['state_code'] = df_final_output['state'].apply(get_state_code)

state_index = df_final_output.columns.get_loc('state') + 1
df_final_output.insert(state_index, 'state_code', df_final_output.pop('state_code'))

print("\Results with all locations:")
print(tabulate(df_final_output, headers='keys', tablefmt='pretty'))

\Results with all locations:
+----+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------+--------------+------------+--------+------------+-----------------------------------------------------------------------------------+--------------------------------+
|    |              Category               |                                                                                          Details                                                                                           |                 Extracted_Location                  |    state     | state_code | county |    city    |                                   climatic_zone                                   |           locations            |
+----+-------------------------------------+-----------------

In [ ]:
df_final_output.to_csv('/content/drive/MyDrive/FYP /AI Alignment/Test/data/preprocessing.csv')